In [6]:
# import required packages
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation

from keras import optimizers
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

from keras.utils import np_utils

In [7]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(
        rescale=1./255,                    
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
        )
validation_data_gen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
        )



# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        '/Users/roma/Desktop/Emotional_recognition/Data/FER 2013 default/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical'
        )

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        '/Users/roma/Desktop/Emotional_recognition/Data/FER 2013 default/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical'
        )

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [8]:
def build_net(optim):
    net = Sequential(name='DCNN')

    net.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            input_shape=(48, 48, 1),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_1'
        )
    )
    net.add(BatchNormalization(name='batchnorm_1'))
    net.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_2'
        )
    )
    net.add(BatchNormalization(name='batchnorm_2'))
    
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
    net.add(Dropout(0.4, name='dropout_1'))

    net.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_3'
        )
    )
    net.add(BatchNormalization(name='batchnorm_3'))
    net.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_4'
        )
    )
    net.add(BatchNormalization(name='batchnorm_4'))
    
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
    net.add(Dropout(0.4, name='dropout_2'))

    net.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_5'
        )
    )
    net.add(BatchNormalization(name='batchnorm_5'))
    net.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_6'
        )
    )
    net.add(BatchNormalization(name='batchnorm_6'))
    
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_3'))
    net.add(Dropout(0.5, name='dropout_3'))

    net.add(Flatten(name='flatten'))
        
    net.add(
        Dense(
            128,
            activation='elu',
            kernel_initializer='he_normal',
            name='dense_1'
        )
    )
    net.add(BatchNormalization(name='batchnorm_7'))
    
    net.add(Dropout(0.6, name='dropout_4'))
    
    net.add(
        Dense(7,
            activation='softmax',
            name='out_layer'
        )
    )
    
    net.compile(
        loss='categorical_crossentropy',
        optimizer= optim,
        metrics=['accuracy']
    )
    
    net.summary()
    
    return net


early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [9]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(
        rescale=1./255,                    
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
        )
validation_data_gen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
        )

In [10]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        '/Users/roma/Desktop/Final Project/Model 3/Data/FER 2013 default/train', # your path of FER 2013 Dataset
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical'
        )

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        '/Users/roma/Desktop/Final Project/Model 3/Data/FER 2013 default/test', # your path of FER 2013 Dataset
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical'
        )

Found 7215 images belonging to 1 classes.
Found 7178 images belonging to 7 classes.


In [4]:
# Train the neural network/model
optims = [
    optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam'),
    optimizers.Adam(0.001),
]
model = build_net(optims[1])
emotion_model_info = model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=200,
        validation_data=validation_generator,
        validation_steps=7178 // 64,
        callbacks=callbacks)

Model: "DCNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        1664      
                                                                 
 batchnorm_1 (BatchNormaliza  (None, 48, 48, 64)       256       
 tion)                                                           
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 64)        102464    
                                                                 
 batchnorm_2 (BatchNormaliza  (None, 48, 48, 64)       256       
 tion)                                                           
                                                                 
 maxpool2d_1 (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 dropout_1 (Dropout)         (None, 24, 24, 64)        0      

2023-04-04 11:32:26.947499: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/cb/nv527ffd4sn4f92077l2twqr0000gn/T/ipykernel_1383/1936430731.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = model.fit_generator(


Epoch 1/200
448/448 [==============================] - 461s 1s/step - loss: 1.9658 - accuracy: 0.3094 - val_loss: 1.5871 - val_accuracy: 0.4015 - lr: 0.0010
Epoch 2/200
448/448 [==============================] - 465s 1s/step - loss: 1.4796 - accuracy: 0.4324 - val_loss: 1.3697 - val_accuracy: 0.4708 - lr: 0.0010
Epoch 3/200
448/448 [==============================] - 459s 1s/step - loss: 1.3427 - accuracy: 0.4910 - val_loss: 1.2492 - val_accuracy: 0.5243 - lr: 0.0010
Epoch 4/200
448/448 [==============================] - 471s 1s/step - loss: 1.2647 - accuracy: 0.5198 - val_loss: 1.2057 - val_accuracy: 0.5453 - lr: 0.0010
Epoch 5/200
448/448 [==============================] - 464s 1s/step - loss: 1.2119 - accuracy: 0.5452 - val_loss: 1.1372 - val_accuracy: 0.5677 - lr: 0.0010
Epoch 6/200
448/448 [==============================] - 462s 1s/step - loss: 1.1638 - accuracy: 0.5630 - val_loss: 1.1324 - val_accuracy: 0.5696 - lr: 0.0010
Epoch 7/200
448/448 [==============================] - 463

In [5]:
# save model structure in jason file
model_json = model.to_json()
with open("emotion_model_4.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
model.save_weights('emotion_model_4.h5')